In [1]:
import os
import jieba
import jieba.analyse
import pandas as pd
import pyodbc
import configparser
from bs4 import BeautifulSoup
import re
import nltk

In [2]:
config = configparser.ConfigParser()
config.read('config.env')
db_UserName = config.get('DEFAULT', 'DB_USERNAME')
db_Password = config.get('DEFAULT', 'DB_PASSWORD')
db_Name = config.get('DEFAULT', 'DB_NAME')
db_Host = config.get('DEFAULT', 'DB_HOST')

cnxn_str = ("Driver={ODBC Driver 17 for SQL Server};"
            f"Server={db_Host};"
            f"Database={db_Name};"
            f"UID={db_UserName};"
            f"PWD={db_Password};")

cnxn = pyodbc.connect(cnxn_str)
# Create a cursor from the connection
cursor = cnxn.cursor()

In [3]:
# SQL查詢語句
query = ("select id,title ,context from ("
         "select a.id,title,context from pttpost_referendum_2 a "
         " inner join pttpost b on a.source=b.source and a.id=b.Id "
         " where not exists (select * from keyword where source=99 and (b.title like '%'+keyname+'%' or b.context like '%'+keyname+'%')) "
         " union all "
         " select a.id,title,context from pttpost_referendum_2 a "
         " inner join pttpostgossing b on a.source=b.source and a.id=b.Id "
         " where not exists (select * from keyword where source=99 and (b.title like '%'+keyname+'%' or b.context like '%'+keyname+'%')) "
         " union all "
         " select convert(varchar,a.id),title,content from dcard.dbo.pttpost_referendum_2 a "
         " inner join dcard.dbo.post b on a.source=b.forum and a.id=b.Id "
         " where not exists (select * from keyword where source=99 and (b.title like '%'+keyname+'%' or b.content like '%'+keyname+'%')) "
         " ) m "
         "where 1=1")

In [4]:
# 讀取資料表
df = pd.read_sql(query, cnxn)

c:\Users\HuanChen\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
# 設置TF-IDF參數
topK = 50
withWeight = True

# 載入知網詞庫
jieba.set_dictionary('C:\project\python\dict.big5.txt')

# 載入自定義詞庫
jieba.load_userdict('C:\project\python\main.txt')

# 設置停用詞
jieba.analyse.set_stop_words('C:\project\python\stopWord2.txt')

# 讀取停用詞表
with open('C:\project\python\stopWord2.txt', 'r', encoding='utf-8') as f:
    stop_words = f.read().split()



Building prefix dict from C:\project\python\dict.big5.txt ...
Loading model from cache C:\Users\HuanChen\AppData\Local\Temp\jieba.u7bf78fb8a3e5c528afaa2a9a1de33675.cache
Loading model cost 0.767 seconds.
Prefix dict has been built successfully.


In [6]:
def PrintKeyWord(col1, col2, resultString):
 # 取出所有關鍵詞
    print('get all key words')
    keywords = []
    for index, row in df.iterrows():
        for keyword, weight in row[col1]:
            keywords.append((keyword, weight))
        for keyword, weight in row[col2]:
            keywords.append((keyword, weight))

    # 轉換為dataframe
    print('轉換為dataframe')
    keywords_df = pd.DataFrame(keywords, columns=['keyword', 'weight'])

    # 合併相同的關鍵詞，計算權重總和
    keywords_grouped = keywords_df.groupby(
        ['keyword']).agg({'weight': 'sum'}).reset_index()

    # 按權重從大到小排序
    print('按權重從大到小排序')
    keywords_sorted = keywords_grouped.sort_values('weight', ascending=False)

    # 取出前50個關鍵詞
    print('取出前50個關鍵詞')
    top_keywords = keywords_sorted.head(50)['keyword'].tolist()

    # 輸出結果
    print(f'{resultString} result:')
    print(top_keywords)

In [7]:
def pagerank(graph, weight=None, alpha=0.85, max_iter=100, tol=1e-6, weight_args=None):
    # 初始化權重
    if weight is None:
        weight = uniform_weight
    # 初始化分數
    scores = {node: 1.0 / len(graph) for node in graph}
    # 開始迭代
    for _ in range(max_iter):
        # 計算每個節點的分數
        new_scores = {}
        for node in graph:
            new_score = 0.0
            for neighbor in graph[node]:
                weight_value = weight(node, neighbor, graph, weight_args)
                new_score += weight_value * scores[neighbor]
            new_scores[node] = new_score
        # 計算調整因子
        sum_diff = sum(abs(new_scores[node] - scores[node]) for node in graph)
        if sum_diff < tol:
            break
        # 更新分數
        for node in graph:
            scores[node] = alpha * new_scores[node] + (1 - alpha) / len(graph)
    return scores

In [8]:
# 定義權重函數
def uniform_weight(x, y, graph, weight_args):
    return 1.0 / len(graph[x])


In [9]:
# 定義 text-rank 分析函數
def get_keywords_textrank(content):
    # 使用 jieba 進行斷詞
    words = jieba.lcut(content)
    # 去除停用詞和非中文詞
    words = [word for word in words if word not in stop_words and re.match(
        '^[\u4e00-\u9fa5]+$', word)]

    # 建立關鍵詞圖
    graph = {}
    for i in range(len(words)):
        if words[i] not in graph:
            graph[words[i]] = set()
        for j in range(i+1, len(words)):
            if words[j] not in graph:
                graph[words[j]] = set()
            if j - i > 5:
                break
            graph[words[i]].add(words[j])
            graph[words[j]].add(words[i])
    # 計算關鍵詞權重
    scores = pagerank(graph, weight=None, alpha=0.85,
                      max_iter=100, tol=1e-6, weight_args=None)
    # 取得前 topK 個權重最大的關鍵詞
    tr_keywords = []
    for word, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)[:topK]:
        tr_keywords.append((word, score))
    return tr_keywords


In [10]:
# 定義處理字串的函式
def process_text(text):
    # 使用 jieba 分詞
    words = jieba.cut(text)
    # 去除停用詞
    words = [word for word in words if word not in stop_words]
    # 回傳字詞列表
    return words

In [11]:
# 去除 HTML tag
print('remove html tag')
df['context'] = df['context'].apply(
    lambda x: BeautifulSoup(x, "html.parser").get_text())
df['title'] = df['title'].apply(
    lambda x: BeautifulSoup(x, "html.parser").get_text())

remove html tag


c:\Users\HuanChen\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
c:\Users\HuanChen\AppData\Local\Programs\Python\Python310\lib\site-packages\bs4\__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(


In [12]:
# 去除特殊符号
print('remove special word')
df['context'] = df['context'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['title'] = df['title'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

remove special word


In [13]:
# 去除HTML tag
print('去除HTML tag')
df['context'] = df['context'].apply(lambda x: re.sub(r'<[^<]+?>', '', x))
df['title'] = df['title'].apply(lambda x: re.sub(r'<[^<]+?>', '', x))

去除HTML tag


In [14]:
# 去除標點符號及數字
print('去除標點符號及數字')
df['context'] = df['context'].apply(
    lambda x: re.sub(r'[^\u4e00-\u9fa5]+', '', x))
df['title'] = df['title'].apply(lambda x: re.sub(r'[^\u4e00-\u9fa5]+', '', x))

去除標點符號及數字


In [15]:
# 去除停用词
print('remove stop words')
df['context'] = df['context'].apply(lambda x: ' '.join(
    [word for word in jieba.analyse.extract_tags(x, topK=topK, withWeight=False) if word not in stop_words]))
df['title'] = df['title'].apply(lambda x: ' '.join([word for word in jieba.analyse.extract_tags(
    x, topK=topK, withWeight=False) if word not in stop_words]))

remove stop words


In [16]:
print('start analyze title stop word')
# 使用jieba對每個記錄的title欄位進行中文斷詞
df['title_cut'] = df['title'].apply(lambda x: ' '.join(jieba.cut(x)))

start analyze title stop word


In [17]:
print('start analyze context  stop word')
# 使用jieba對每個記錄的context欄位進行中文斷詞
df['context_cut'] = df['context'].apply(lambda x: ' '.join(jieba.cut(x)))

start analyze context  stop word


In [18]:
print('start analyze title tf-idf')
# 使用tf-idf算法計算每個記錄的title欄位的關鍵詞
df['title_keywords'] = df['title_cut'].apply(
    lambda x: jieba.analyse.extract_tags(x, topK=topK, withWeight=withWeight))

start analyze title tf-idf


In [19]:
print('start analyze context tf-idf')
# 使用tf-idf算法計算每個記錄的context欄位的關鍵詞
df['context_keywords'] = df['context_cut'].apply(
    lambda x: jieba.analyse.extract_tags(x, topK=topK, withWeight=withWeight))

start analyze context tf-idf


In [20]:
PrintKeyWord('title_keywords', 'context_keywords','tf-idf')

get all key words
轉換為dataframe
按權重從大到小排序
取出前50個關鍵詞
tf-idf result:
['問卦', '新聞', '台灣', '八卦', '中國', '爆卦', '公投', '黑特', '美國', '核四', '蔡英文', '社會', '發文', '總統', '柯文哲', '台灣人', '相關', '個月', '也算', '塔綠班', '前請', '民眾', '國家', '媒體', '鄉民', '記者', '政治', '網軍', '萊豬', '高雄', '禁止', '四個', '不同意', '缺電', '中國人', '發電', '政府', '館長', '請注意', '核廢料', '藻礁', '東西', '台北', '風向', '中共', '台電', '未來', '新聞標題', '立委', '面對']


In [21]:
# 將 text-rank 分析結果加入 DataFrame 中
print('start analyze context textrank')
# df['tr_content_keywords'] = df['context_cut'].apply(get_keywords_textrank)
df['tr_content_keywords'] = df['context_cut'].apply(lambda x: jieba.analyse.textrank(
    x, topK=topK, withWeight=withWeight, allowPOS=('ns', 'n', 'vn', 'v')))


start analyze context textrank


In [22]:
print('start analyze title textrank')
# df['tr_title_keywords'] = df['title_cut'].apply(get_keywords_textrank)
df['tr_title_keywords'] = df['title_cut'].apply(
    lambda x: jieba.analyse.textrank(x, topK=topK, withWeight=withWeight, allowPOS=('ns', 'n', 'vn', 'v')))


start analyze title textrank


In [23]:
PrintKeyWord('tr_content_keywords', 'tr_title_keywords','text-rank')

get all key words
轉換為dataframe
按權重從大到小排序
取出前50個關鍵詞
text-rank result:
['八卦', '問卦', '政治', '禁止', '政府', '好像', '公投', '日本', '台北', '朋友', '不用', '人民', '支持', '事情', '影片', '小孩', '民主', '投票', '事件', '希望', '文章', '世界', '政策', '地方', '公司', '工作', '小弟', '垃圾', '分享', '不到', '台北市', '代表', '核能', '安安', '原本', '道歉', '人生', '留言', '方式', '肥宅', '想到', '生活', '上班', '能力', '想法', '理由', '能源', '名字', '父母', '原因']


In [24]:
print('start analyze title tf')
# 處理 context 欄位
corpus_context = [process_text(text) for text in df['context']]
# 處理 title 欄位
corpus_title = [process_text(text) for text in df['title']]
# 合併兩個 corpus
corpus = corpus_context + corpus_title

# 使用 nltk.FreqDist 計算詞頻
word_freq = nltk.FreqDist(word for words in corpus for word in words)
# 取出前 50 筆(去掉空格多取一筆)
top_words = word_freq.most_common(51)

# 顯示結果
for word, freq in top_words:
    # print(f"{word}: {freq}")
    print(f"{word}")


start analyze title tf
 
問卦
新聞
台灣
八卦
中國
政治
記者
發文
美國
相關
媒體
個月
社會
完整
禁止
也算
前請
政府
新聞標題
公投
國家
台灣人
總統
民眾
報導
核四
鄉民
蔡英文
東西
請注意
日本
好像
也有
自己的
台北
柯文哲
中共
一點
朋友
不用
我覺得
塔綠班
爆卦
都會
未來
高雄
就會
同意
面對
支持
